## Preprocessing

In [69]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [70]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [71]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [72]:
# Look at APPLICATION_TYPE value counts for binning
type_count = application_df['APPLICATION_TYPE'].value_counts()
type_count

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [73]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

application_types_to_replace = type_count[type_count<1000].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [74]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [75]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count = application_df['CLASSIFICATION'].value_counts()
class_count[class_count > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [76]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_to_replace = class_count[class_count<1000].index.tolist()

# Replace in dataframe
for cls in class_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [77]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [78]:
#define model
def create_model(learning_rate=0.01):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


In [79]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values

# drop 'IS SUCCESSFUL'
X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [81]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 =  50
hidden_nodes_layer2 = 45
hidden_nodes_layer3 = 30

#optimizer = RMSprop(learning_rate=0.001, decay=1e-6)
learning_rate = 0.01 
optimizer = SGD(learning_rate=learning_rate)

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
nn.add(tf.keras.layers.Dropout(0.325))
# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu")
)
nn.add(tf.keras.layers.Dropout(0.325))

# Test 3rd layer
#nn.add(
    #tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
#nn.add(tf.keras.layers.Dropout(0.325))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/Users/katiestarnes/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_32 (Dense)                │ (None, 50)             │         2,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 45)             │         2,295 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 45)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 1)              │            46 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,391 (17.15 KB)

 Trainable params: 4,391 (17.15 KB)

 Non-trainable params: 0 (0.00 B)

In [82]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [83]:
# Train the model
#add validation split
#add batch size
fit_model = nn.fit(X_train_scaled, y_train, validation_split=0.155, epochs=75, batch_size = 32)

Epoch 1/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 1s 442us/step - accuracy: 0.5454 - loss: 0.7286 - val_accuracy: 0.7097 - val_loss: 0.6106
Epoch 2/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 367us/step - accuracy: 0.6717 - loss: 0.6332 - val_accuracy: 0.7318 - val_loss: 0.5797
Epoch 3/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - accuracy: 0.6914 - loss: 0.6117 - val_accuracy: 0.7339 - val_loss: 0.5679
Epoch 4/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 368us/step - accuracy: 0.7006 - loss: 0.6045 - val_accuracy: 0.7358 - val_loss: 0.5680
Epoch 5/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step - accuracy: 0.7018 - loss: 0.5995 - val_accuracy: 0.7384 - val_loss: 0.5610
Epoch 6/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step - accuracy: 0.7050 - loss: 0.5957 - val_accuracy: 0.7381 - val_loss: 0.5590
Epoch 7/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - accuracy: 0.7110 - loss: 0.5880 - val_accuracy: 0.7377 - val_loss: 0.5589
Epoch 8/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 379us/step - accuracy: 0.7100 - loss: 0.5861 - 

725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step - accuracy: 0.7250 - loss: 0.5666 - val_accuracy: 0.7388 - val_loss: 0.5520
Epoch 51/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - accuracy: 0.7275 - loss: 0.5611 - val_accuracy: 0.7384 - val_loss: 0.5517
Epoch 52/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step - accuracy: 0.7243 - loss: 0.5651 - val_accuracy: 0.7388 - val_loss: 0.5514
Epoch 53/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step - accuracy: 0.7292 - loss: 0.5630 - val_accuracy: 0.7391 - val_loss: 0.5518
Epoch 54/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - accuracy: 0.7228 - loss: 0.5686 - val_accuracy: 0.7388 - val_loss: 0.5511
Epoch 55/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 447us/step - accuracy: 0.7240 - loss: 0.5623 - val_accuracy: 0.7393 - val_loss: 0.5518
Epoch 56/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.7239 - loss: 0.5631 - val_accuracy: 0.7330 - val_loss: 0.5521
Epoch 57/75
725/725 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step - accuracy: 0.7306 - loss: 0.5616 - val_

In [84]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 308us/step - accuracy: 0.7306 - loss: 0.5556
Loss: 0.5555679202079773, Accuracy: 0.7306122183799744


In [85]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimizationy.h5")